In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la
import scipy.sparse
import scipy.sparse.linalg as sla
import pickle

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

import codes.higher_order_lowdin as lowdin
from codes.combine import apply_peierls_to_template, magnetic_perturbation

In [ ]:
# %load_ext line_profiler

# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/semicon/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import kwant
import scipy
import sympy
import semicon


print("semicon version:", semicon.__version__)
print("kwant version:", kwant.__version__)
print("scipy version:", scipy.__version__)
print("sympy version", sympy.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=kwant.KwantDeprecationWarning)

## Prepare k.p model and solve it exactly

In [ ]:
gamma_0 = 1.0

# Get the k.p model for ZincBlende material
model = semicon.models.ZincBlende(
    components=('foreman',),
    default_databank='lawaetz'
)
kpham = model.hamiltonian

# Get parameters for InAs
InAs = model.parameters('InAs').renormalize(new_gamma_0=gamma_0)

display(kpham)

# Add Zeeman field with InAs bulk g-factor value
g_bulk = -15

kpham += sum(sympy.sympify(B) * g_bulk * sympy.sympify('mu_B') * model.spin_operators[i] for i, B in enumerate(['B_x', 'B_y', 'B_z']))

# Add electric field
kpham += sympy.sympify('- E_x * x - E_y * y') * np.eye(8)

InAs = {**InAs, 'B_x': 0, 'B_y': 0, 'B_z': 0, 'E_x': 0, 'E_y': 0}

In [ ]:
kpham

In [ ]:
# Discretize the model to get a kwant tight-binding model
grid_spacing = 0.5
R = 10

tbham = kwant.continuum.discretize(kpham, grid=grid_spacing)
# Add orbital magnetic field
tbham = apply_peierls_to_template(tbham)

# Fill an infinite wire with circular cross section with the model
shape = lambda site: la.norm(site.pos) < R
tz = np.array(tbham.symmetry.periods)[-1]
wire = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
wire.fill(tbham, shape, start=(0, 0, 0))
kwant.plotter.plot(wire, num_lead_cells=10, pos_transform=lambda pos: pos[:2]);

# Get the unit cell and hopping matrices
wiref = wire.finalized()
Hos = wiref.cell_hamiltonian(params=InAs, sparse=True).tocsr()
Hos.eliminate_zeros()
Hhop = wiref.inter_cell_hopping(params=InAs, sparse=True).tocsr()
Hhop.eliminate_zeros()

# Make second order model in k_z
H0 = lowdin.Model({1: Hos + (Hhop + Hhop.T.conj())})
Hkz = lowdin.Model({'k_z': (1j * Hhop - 1j * Hhop.T.conj()), 'k_z**2': -(Hhop + Hhop.T.conj())})
H = H0 + Hkz

In [ ]:
def exact_spectrum(wire, ks, params, sigma=0.5, num_states=6):
    wrapped_wire = kwant.wraparound.wraparound(wire).finalized()
    es = []
    for k in ks:
        pars = {**params, 'k_x': k}
        e = scipy.sparse.linalg.eigsh(wrapped_wire.hamiltonian_submatrix(params=pars, sparse=True),
                                      return_eigenvectors=False, sigma=sigma, k=num_states)
        e = sorted(e)
        es.append(e)
    return np.array(es)

In [ ]:
# exact eigenenergies
ks = np.linspace(-0.1, 0.1, 21)

# This takes long, precalculate it
try:
    es = pickle.load(open('wire_exact_0.pickle', 'rb'))['es']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z)
    es = exact_spectrum(wire, ks, InAs, sigma=0.5, num_states=6)
    pickle.dump(dict(es=es,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open('wire_exact_0.pickle', 'wb'))
    
plt.plot(ks, es, '-');

## Build perturbative Hamiltonian

In [ ]:
def electric_terms(wiref):
    # Make electric field perturbations

    xop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[0] * np.eye(8), sum=True).tocoo().tocsr()
    xop.eliminate_zeros()
    yop = kwant.operator.Density(wiref, onsite=lambda site: site.pos[1] * np.eye(8), sum=True).tocoo().tocsr()
    yop.eliminate_zeros()

    V = lowdin.Model({'E_x': -xop, 'E_y': -yop})
    
    return V

def magnetic_terms(tbham, shape):
    # Make magnetic field perturbations
    HB = 0
    for i, (n, B) in enumerate(zip(np.eye(3), ['B_x', 'B_y', 'B_z'])):
        # orbital magnetic field
        tbhamB = magnetic_perturbation(tbham, n=n)
        wireB = kwant.Builder(symmetry=kwant.TranslationalSymmetry(tz))
        wireB.fill(tbhamB, shape, start=(0, 0, 0))
        wireB = wireB.finalized()
        matB = wireB.cell_hamiltonian(params=InAs, sparse=True).tocsr()
        matB += wireB.inter_cell_hopping(params=InAs, sparse=True).tocsr() + wireB.inter_cell_hopping(params=InAs, sparse=True).tocsr().T.conj()
        HB += lowdin.Model({B: matB})
        # Zeeman term
        matZ = kwant.operator.Density(wiref, onsite=lambda site: model.spin_operators[i], sum=True).tocoo().tocsr()
        matZ.eliminate_zeros()
        HB += lowdin.Model({B: InAs['mu_B'] * g_bulk * matZ})

    return HB

In [ ]:
sz0 = 0.5 * np.diag([1, -1, 3, 1, -1, -3, -1, 1])

sz = kwant.operator.Density(wiref, 
                            onsite=lambda site: sz0,
                            sum=True).tocoo().tocsr()

def fix_basis(E0, vecsA):
    # Order eigenstates by energy
    e_order = np.argsort(E0)
    E02 = np.sort(E0)
    vecsA2 = vecsA[:, e_order]
    # Make sure that the degenerate states are orthogonal
    vecsA2, _ = la.qr(vecsA2, mode='economic')
    assert len(E02) % 2 == 0
    # Fix basis in degenerate eigensubspace by spin z polarization
    szA = vecsA2.T.conj() @ sz @ vecsA2
    for i in range(0, len(E02), 2):
        assert np.isclose(E02[i], E02[i+1]), (i, E02)
        _, U = la.eigh(szA[i:i+2, i:i+2])
        vecsA2[:, i:i+2] = vecsA2[:, i:i+2] @ U
        szAi = vecsA2[:, i:i+2].T.conj() @ sz @ vecsA2[:, i:i+2]
        assert np.allclose(szAi, np.diag(np.diag(szAi)))
    # fix phases, this is arbitrary, but removes phase ambiguity
    phases = np.diag(np.angle(vecsA2.T @ vecsA2))
    vecsA2 = vecsA2 @ np.diag(np.exp(-1j*phases/2))
    return E02, vecsA2

### Lowest subband $l=0$ states

In [ ]:
# Make perturbation basis of two lowest states
kpm_params = dict(num_moments=1000)

E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.45, k=2)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)

V = electric_terms(wiref)
HB = magnetic_terms(tbham, shape)

# only keep most interesting keys
intersting_keys = sympy.sympify(['k_z', 'k_z**2',
                                 'E_x', 'E_y', 'E_x**2', 'E_y**2',
                                 'B_x', 'B_y', 'B_z',
                                 'E_x * k_z', 'E_y * k_z'])

model0 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA, order=2,
                                kpm_params=kpm_params,
                                interesting_keys=intersting_keys)

model0.around(decimals=4).tosympy()

In [ ]:
# effective eigenenergies with electric and magnetic fields
es0 = []
E_x = 0.02 # V/nm
B_z = 0.5 # Tesla
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es0 = np.array(es0)

# This takes long, precalculate it
try:
    esEB = pickle.load(open('wire_exact.pickle', 'rb'))['esEB']
except:
    params = InAs.copy()
    params.update(E_x=E_x, B_z=B_z)
    esEB = exact_spectrum(wire, ks, params, sigma=0.42, num_states=2)
    pickle.dump(dict(esEB=esEB,
                     params=params,
                     R=R,
                     grid_spacing=grid_spacing),
        open('wire_exact.pickle', 'wb'))
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es0, '-')

In [ ]:
plt.plot(ks, esEB - 0.0248, '--')
plt.plot(ks, es0, '-')
plt.ylim([0.375, 0.39])
plt.xlim([-0.05, 0.05])

In [ ]:
# Use more states exactly
kpm_params = dict(num_moments=100)

# Make perturbation basis of two lowest states
E0, vecs = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.4, k=60)
E0, vecs = fix_basis(E0, vecs)
print(E0)

ind_0 = np.argwhere(E0 > 0)[:2].flatten()

model0 = lowdin.effective_model(H0, Hkz + V + HB,
                                evec_A=vecs[:, ind_0],
                                evec_B=vecs[:, np.setdiff1d(np.arange(vecs.shape[1]), ind_0)],
                                order=2, interesting_keys=intersting_keys,
                                kpm_params=kpm_params)
display(model0.around(decimals=4).tosympy())

In [ ]:
# effective eigenenergies with electric and magnetic fields
es0 = []
E_x = 0.02 # V/nm
B_z = 0.5 # Tesla
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model0).subs(pars)[1], eigvals_only=True)
    es0.append(e)
es0 = np.array(es0)
    
plt.plot(ks, esEB, '--')
plt.plot(ks, es0, '-')

In [ ]:
# Shift exact spectrum to match it
plt.plot(ks, esEB - 0.0248, '--')
plt.plot(ks, es0, '-')
plt.ylim([0.375, 0.39])
plt.xlim([-0.05, 0.05])

### Second lowest subband $l=1$ states

In [ ]:
# Make perturbation basis of second lowest states
kpm_params = dict(num_moments=1000)
E0, vecsA = scipy.sparse.linalg.eigsh(H0[1], return_eigenvectors=True, sigma=0.55, k=4)
E0, vecsA = fix_basis(E0, vecsA)
print(E0)
print(np.diag(vecsA.T.conj() @ sz @ vecsA))
print(np.diag(vecsA.T.conj() @ Hos @ vecsA))

model1 = lowdin.effective_model(H0, Hkz + V + HB, evec_A=vecsA,
                                order=2, interesting_keys=intersting_keys,
                                kpm_params=kpm_params)

model1.around(decimals=4).tosympy()

In [ ]:
# effective eigenenergies with zero fields
es1 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': 0, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': 0}
    e = la.eigh((model1).subs(pars)[1], eigvals_only=True)
    es1.append(e)
es1 = np.array(es1)
    
plt.plot(ks, es1, '-')

In [ ]:
# effective eigenenergies with electric field
E_x = 0.01
es1 = []
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': 0}
    e = la.eigh((model1).subs(pars)[1], eigvals_only=True)
    es1.append(e)
es1 = np.array(es1)

plt.plot(ks, es1, '-')

In [ ]:
# effective eigenenergies with electric and magnetic fields
es1 = []
E_x = 0.02
B_z = 0.5
ks = np.linspace(-0.1, 0.1, 101)
for k in ks:
    pars = {'k_z': k, 'E_x': E_x, 'E_y': 0, 'B_x': 0, 'B_y': 0, 'B_z': B_z}
    e = la.eigh((model1).subs(pars)[1], eigvals_only=True)
    es1.append(e)
es1 = np.array(es1)

plt.plot(ks, es1, '-')
plt.ylim([0.48, 0.50])